In [ ]:
# PHASE 1: MODEL TRAINING AND SAVING

# IMPORTS
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from collections import Counter
import shutil
import random
from google.colab import drive
import matplotlib.pyplot as plt

# CLASS CONFIGURATIONS
CLASSES = ['COVID', 'NORMAL', 'PNEUMONIA', 'TUBERCULOSIS']
NUM_CLASSES = len(CLASSES)
IMG_SIZE = 224
BATCH_SIZE = 32
BASE_DIR = 'data_split'

# Drive mounting and path definitions
drive.mount('/content/drive')
DRIVE_PATH = '/content/drive/MyDrive/NNDL_CXR_Project' # Folder for saved model
MODEL_FILENAME = os.path.join(DRIVE_PATH, 'resnet_cxr_multidisease_model.h5')
RAW_DATA_PATH = '/content/drive/MyDrive/xray_img' # INPUT DATASET
os.makedirs(DRIVE_PATH, exist_ok=True) # Ensure the output directory for the model exists

# 1)Data Preparation and Splitting

def setup_data_directories():
    for split in ['train', 'val', 'test']:
        for class_name in CLASSES:
            os.makedirs(os.path.join(BASE_DIR, split, class_name), exist_ok=True)
    print("Required data directories (train/val/test) created in:", BASE_DIR)

def split_data_into_folders(raw_data_dir, classes, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    print("\n--- Starting Data Split (70/15/15) ---")
    setup_data_directories()

    if os.path.exists(os.path.join(BASE_DIR, 'train', classes[0])) and len(os.listdir(os.path.join(BASE_DIR, 'train', classes[0]))) > 5:
        print("Data already appears to be split and copied. Skipping split function.")
        return

    for class_name in classes:
        source_dir = os.path.join(raw_data_dir, class_name)
        if not os.path.exists(source_dir):
            print(f"FATAL ERROR: Source directory not found for class '{class_name}'.")
            print(f"Please check your Google Drive path: {source_dir}")
            return

        all_files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not all_files:
            print(f"No image files found in {source_dir}. Skipping class.")
            continue

        train_files, temp_files = train_test_split(all_files, train_size=train_ratio, random_state=42)
        val_files, test_files = train_test_split(temp_files, test_size=test_ratio/(val_ratio + test_ratio), random_state=42)

        print(f"Class {class_name}: Total={len(all_files)}, Train={len(train_files)}, Val={len(val_files)}, Test={len(test_files)}")

        for f in train_files:
            shutil.copy(os.path.join(source_dir, f), os.path.join(BASE_DIR, 'train', class_name, f))
        for f in val_files:
            shutil.copy(os.path.join(source_dir, f), os.path.join(BASE_DIR, 'val', class_name, f))
        for f in test_files:
            shutil.copy(os.path.join(source_dir, f), os.path.join(BASE_DIR, 'test', class_name, f))

    print("--- Data Splitting and Copying Complete ---")

# Executing Data Split
split_data_into_folders(RAW_DATA_PATH, CLASSES)

# ImageDataGenerator setup
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Generator Execution
train_generator = train_datagen.flow_from_directory(
    os.path.join(BASE_DIR, 'train'), target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', classes=CLASSES,
)
val_generator = val_test_datagen.flow_from_directory(
    os.path.join(BASE_DIR, 'val'), target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', classes=CLASSES,
)

# 2)Model Definition and Class Weight Calculation

def calculate_class_weights(generator):
    counter = Counter(generator.classes)
    total = float(sum(counter.values()))
    max_frequency = float(max(counter.values()))

    # Calculate initial weights based on inverse class frequency
    initial_weights = {cls_idx: total / (len(counter) * count) for cls_idx, count in counter.items()}

    # Normalizing weights
    max_weight = max(initial_weights.values())
    class_weights = {cls_idx: weight / max_weight for cls_idx, weight in initial_weights.items()}

    # Scale all weights by a factor (e.g., 2 or 3) to emphasize rare classes more
    scaling_factor = 2.0
    class_weights = {cls_idx: weight * scaling_factor for cls_idx, weight in class_weights.items()}

    print(f"\n--- Class Distribution (by index: {generator.class_indices}) ---")
    for cls_idx, count in counter.items():
        print(f"Class {CLASSES[cls_idx]} ({cls_idx}): {count} samples. Weight: {class_weights[cls_idx]:.2f}")

    return class_weights

def build_resnet_model(num_classes, trainable_layers=0, learning_rate=1e-3):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

    # Set trainable layers based on the number of layers in the model
    if trainable_layers == 0:
        for layer in base_model.layers:
            layer.trainable = False
    else:
        # Unfreeze the top layers for fine-tuning
        for layer in base_model.layers[:-trainable_layers]:
            layer.trainable = False
        for layer in base_model.layers[-trainable_layers:]:
            layer.trainable = True

    # Check if the head needs to be added (only add once)
    if not base_model.output.shape[-1] == num_classes:
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(num_classes, activation='softmax', name='predictions')(x)
        model = Model(inputs=base_model.input, outputs=predictions)
    else:
        model = base_model

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy',
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall(),
                           tf.keras.metrics.AUC(name='auc')])
    return model

# 3)Two-Stage Training and Saving Model

def train_and_save_model():

    class_weights = calculate_class_weights(train_generator)

    # STAGE 1: Train Classification Head (Fast Learning Rate)
    print("\n\n--- STAGE 1: Training Classification Head (Fast) ---")
    model = build_resnet_model(NUM_CLASSES, trainable_layers=0, learning_rate=1e-3)

    checkpoint_stage1 = ModelCheckpoint(MODEL_FILENAME, monitor='val_auc', save_best_only=True, mode='max', verbose=1)
    early_stop_stage1 = EarlyStopping(monitor='val_auc', patience=5, restore_best_weights=True, mode='max')

    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=15, # Shorter epochs for head training
        validation_data=val_generator,
        validation_steps=val_generator.samples // BATCH_SIZE,
        callbacks=[checkpoint_stage1, early_stop_stage1],
        class_weight=class_weights # Use weights here to calibrate initial head
    )

    # Loading the best model from Stage 1 before fine-tuning
    best_stage1_model = load_model(MODEL_FILENAME)

    # STAGE 2: Fine-Tuning Last Block (Slow Learning Rate + Weights)
    print("\n\n--- STAGE 2: Fine-Tuning Last 30 Layers (Slow) ---")
    # 30 layers corresponds roughly to the last major block of ResNet50
    layers_to_unfreeze = 30

    # Re-compile the best model from Stage 1 with unfrozen layers and very low LR
    model = build_resnet_model(NUM_CLASSES, trainable_layers=layers_to_unfreeze, learning_rate=1e-5)

    # Transfer weights from the best model of Stage 1
    model.set_weights(best_stage1_model.get_weights())

    # Define new callbacks for Stage 2
    checkpoint_stage2 = ModelCheckpoint(MODEL_FILENAME, monitor='val_auc', save_best_only=True, mode='max', verbose=1)
    early_stop_stage2 = EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True, mode='max')

    print(f"Starting Fine-Tuning for {50} epochs (or until Early Stopping)...")
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=50, # More epochs for fine-tuning
        validation_data=val_generator,
        validation_steps=val_generator.samples // BATCH_SIZE,
        callbacks=[checkpoint_stage2, early_stop_stage2],
        class_weight=class_weights # Using weights to improve balanced accuracy
    )

    print(f"\nTraining Complete. FINAL best model saved persistently to Google Drive: {MODEL_FILENAME}")

# 4)Model Evaluation on Test Set

def evaluate_saved_model_on_test_data():
    print("\n--- Starting Test Set Evaluation ---")
    try:
        best_model = load_model(MODEL_FILENAME)
        print(f"Successfully loaded best model from {MODEL_FILENAME}")
    except Exception as e:
        print(f"ERROR: Could not load the model. Ensure training ran successfully. Error: {e}")
        return

    # Creating Test Generator
    test_generator = val_test_datagen.flow_from_directory(
        os.path.join(BASE_DIR, 'test'),
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        classes=CLASSES,
        shuffle=False
    )

    print("\nEvaluating model on the final, unseen test dataset...")
    metrics = best_model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)

    metric_names = ['Loss', 'Accuracy', 'Precision', 'Recall', 'AUC']
    print("-" * 40)
    print("Final Test Set Performance:")
    print("-" * 40)
    for name, value in zip(metric_names, metrics):
        if name == 'Loss':
            print(f"{name:<10}: {value:.4f}")
        else:
            style = '**' if name in ['Accuracy', 'AUC'] else ''
            print(f"{name:<10}: {style}{value * 100:.2f}%{style}")
    print("-" * 40)

    if metrics[1] < 0.85:
        print("💡 NOTE: Accuracy is still below target. Fine-tuning more layers or using more data augmentation may be needed.")
    else:
        print("✅ SUCCESS: Test Accuracy is high, indicating good model generalization and no major overfitting.")

    print("You can now proceed to Phase 2 (Block 2), even on a CPU runtime.")


# TRAINING AND SAVING EXECUTION
train_and_save_model()

# FINAL TEST EVALUATION
evaluate_saved_model_on_test_data()

In [ ]:
!pip install -q tf-keras-vis gradio

In [ ]:
# MODEL LOADING AND GRADIO INTEGRATION

# IMPORTS
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from google.colab import drive # For mounting drive
import gradio as gr
from PIL import Image
import cv2
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear


# CLASS CONFIGURATION
CLASSES = ['COVID', 'NORMAL', 'PNEUMONIA', 'TUBERCULOSIS']
IMG_SIZE = 224

drive.mount('/content/drive')
DRIVE_PATH = '/content/drive/MyDrive/NNDL_CXR_Project' # Path of saved model
MODEL_FILENAME = os.path.join(DRIVE_PATH, 'resnet_cxr_multidisease_model.h5')

# 4)Grad-CAM Visualisation Module

def visualize_gradcam(model, input_image_array, target_index):
    modifier = ReplaceToLinear()
    def loss_function(output):
        return tf.cast(output[0][target_index], tf.float32)

    # Instantiate Gradcam model and Heat-map computation
    model_for_gradcam = tf.keras.models.clone_model(model)
    model_for_gradcam.set_weights(model.get_weights())

    gradcam = Gradcam(model_for_gradcam, model_modifier=modifier, clone=True)
    cam = gradcam(loss_function, input_image_array, penultimate_layer=-1)
    heatmap = np.uint8(255 * cam[0])

    # Original image preparation for overlay
    img = Image.fromarray((input_image_array[0] * 255).astype(np.uint8))
    img = np.array(img.resize((IMG_SIZE, IMG_SIZE)))

    # Overlaying heatmap onto the original image
    heatmap_colored = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap_colored * 0.4 + img * 0.6 # Blending
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)

    return Image.fromarray(superimposed_img)

# 5)Gradio Interface Logic and Prediction Function

def classify_xray(input_img_pil):
    """
    Handles image processing, prediction, Grad-CAM generation, and output formatting.
    """
    # 1)Load Model
    try:
        # Loading the saved model with the custom AUC metric included
        print(f"Loading model from: {MODEL_FILENAME}")
        model = load_model(MODEL_FILENAME, custom_objects={'auc': tf.keras.metrics.AUC(name='auc')})
    except Exception as e:
        return None, f"<h1>Model Load Error</h1><p>Could not load model from Drive path: '{MODEL_FILENAME}'. **Did you run Phase 1 (Block 1) on GPU and mount Google Drive?** Error: {e}</p>", ""

    # Preprocessing image
    img = input_img_pil.resize((IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0 # Normalizing

    # Prediction Making
    predictions = model.predict(img_array)[0]

    # Getting the top prediction details
    predicted_index = np.argmax(predictions)
    predicted_class = CLASSES[predicted_index]
    confidence = predictions[predicted_index] * 100

    # Generation of Grad-CAM visualization
    gradcam_img = visualize_gradcam(model, img_array, predicted_index)

    # HTML Output and Alert Message(UI Part) Generation

    # Defining Alert Messages based on Predicted Class and Confidence level
    if predicted_class != 'NORMAL' and confidence >= 90:
        alert_style = 'background-color: #ffe5e5; border: 3px solid #f00; color: #a30000; padding: 15px; border-radius: 10px; font-weight: 700; box-shadow: 0 4px 8px rgba(0,0,0,0.1);'
        alert_message = f"🔴 CRITICAL ALERT: High likelihood of {predicted_class} (Confidence: {confidence:.1f}%). Immediate clinical correlation and confirmatory testing are **strongly recommended**."

    elif predicted_class != 'NORMAL' and confidence >= 50:
        alert_style = 'background-color: #fff8e1; border: 3px solid #ffc107; color: #8d6e30; padding: 15px; border-radius: 10px; font-weight: 600;'
        alert_message = f"⚠️ ELEVATED CONCERN: Moderate evidence of lung pathology detected. Moderate likelihood of {predicted_class} (Confidence: {confidence:.1f}%). Suggests a need for close follow-up and specialist review."

    elif predicted_class == 'NORMAL' and confidence >= 90:
        alert_style = 'background-color: #e6f7e6; border: 3px solid #4caf50; color: #2e7d32; padding: 15px; border-radius: 10px;'
        alert_message = f"✅ NO ACUTE PATHOLOGY DETECTED: Model predicts a Normal/Healthy condition (Confidence: {confidence:.1f}%). Please consult a medical professional for final assessment."

    elif predicted_class == 'NORMAL' and confidence >= 50:
        alert_style = 'background-color: #f0fff4; border: 1px solid #90ee90; color: #388e3c; padding: 15px; border-radius: 10px; font-weight: 600;'
        alert_message = f"🟢 Model suggests Moderate likelihood of Normal Condition (Confidence: {confidence:.1f}%). Due to moderate certainty, routine follow-up or a quick clinical is advised."

    else:
        alert_style = 'background-color: #f7fafc; border: 1px solid #e2e8f0; color: #4a5568; padding: 10px; border-radius: 8px;'
        alert_message = "General screening result. Review all scores and consult a medical professional."

    alert_html = f"<div style='{alert_style}'>{alert_message}</div>"

    # 2)Final Results Table(Showing the confidence scores)
    results_html = f"<h2 style='color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px;'>🩺 Multi-Disease Screening Report</h2>"
    results_html += f"<div style='margin-bottom: 25px; padding: 15px; border-radius: 12px; background-color: #3498db; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>"
    results_html += f"<h3 style='margin: 0;'>Primary Diagnosis: {predicted_class} ({confidence:.1f}%)</h3>"
    results_html += "</div>"

    results_html += "<h4 style='color: #2c3e50;'>Confidence Scores:</h4>"
    results_html += "<table style='width: 100%; border-collapse: separate; border-spacing: 0 10px; font-size: 15px;'>"

    scores_list = sorted([(CLASSES[i], p * 100) for i, p in enumerate(predictions)], key=lambda x: x[1], reverse=True)

    for condition, score in scores_list:
        bg_color = '#e9f5ff' if condition == predicted_class else '#f8f9fa'
        text_color = '#2980b9' if condition == predicted_class else '#34495e'
        font_weight = 'bold' if condition == predicted_class else 'normal'

        results_html += f"<tr style='background-color: {bg_color}; border-radius: 8px; box-shadow: 0 1px 3px rgba(0,0,0,0.05);'>"
        results_html += f"<td style='padding: 10px; border-radius: 8px 0 0 8px; color: {text_color}; font-weight: {font_weight};'>{condition}</td>"
        results_html += f"<td style='padding: 10px; border-radius: 0 8px 8px 0; text-align: right; color: {text_color}; font-weight: {font_weight};'>{score:.2f}%</td></tr>"

    results_html += "</table>"

    # 3)Final Disclaimer Section
    disclaimer_html = f"<p style='margin-top: 30px; font-size: 12px; color: #7f8c8d; border-top: 1px dashed #ecf0f1; padding-top: 10px;'>**Disclaimer:** This is a **Computer-Aided Diagnosis (CAD)** tool intended for rapid screening purposes only. **Final medical diagnosis must be made by a qualified radiologist.**</p>"

    final_html_output = alert_html + results_html + disclaimer_html

    return gradcam_img, final_html_output, predicted_class

# 6)Gradio Interface

# I/p & O/p definition
input_image = gr.Image(type="pil", label="Upload Chest X-ray Image")
output_gradcam = gr.Image(type="pil", label="Grad-CAM Visualization (Model Evidence)")
output_html = gr.HTML(label="Diagnostic Report and Alert")
output_text = gr.Text(label="Predicted Class (For Reference)", visible=False)

# Interface definition
iface = gr.Interface(
    fn=classify_xray,
    inputs=input_image,
    outputs=[output_gradcam, output_html, output_text],
    title="🩺 LungAI: Deep Learning-Based Lung Disease Detection",
    description="A Deep Learning system utilizing Convolutional Image Classification and Grad-CAM for the rapid and simultaneous detection of four major lung conditions (Normal, Pneumonia, COVID-19, Tuberculosis).",
    theme="soft"
)

# GRADIO INTERFACE LAUCH
print("\n Project is ready! Loading model and launching Gradio Interface... ")
iface.launch(share=True)